In [2]:
from datetime import datetime
current:datetime = datetime.now()
print(current.year)
print(current.month)
print(current.day)
print(current.hour)
print(current.minute)
print(current.second)

2024
9
4
13
35
9


In [3]:
current.strftime('%d/%m/%Y, %H:%M:%S')

'04/09/2024, 13:35:09'

In [4]:
current.strftime(f'中華民國{current.year-1911}年%m月%d日 %H:%M:%S')

'中華民國113年09月04日 13:35:09'

In [ ]:
import requests
from requests import Response
youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'
try:
    res:Response = requests.request('GET',youbike_url)
    res.raise_for_status()
except Exception:
    print('error')
else:
    content = res.text

from pydantic import BaseModel,RootModel,Field,field_validator,field_serializer###
from datetime import datetime
from pprint import pprint

class Station(BaseModel):
    編碼:str = Field(alias='sno')
    站名:str = Field(alias='sna')
    行政區:str = Field(alias='sarea')
    時間:datetime = Field(alias='mday')
    地點:str = Field(alias='ar')
    狀態:str = Field(alias='act')
    總數:int = Field(alias='total')
    可借數:int = Field(alias='available_rent_bikes')

    @field_validator('站名')
    @classmethod
    def slice_str(cls,name:str)->str:
        return name[11:]
    
    @field_validator('狀態',mode='before')
    @classmethod
    def status_str(cls,status:str)->str:
        if status == '1' :
            return "營運中"
        else:
            return "維護中"
        
    @field_serializer('時間')   ###
    def time(self,date:datetime)->str:
        return date.strftime(f'中華民國{date.year-1911}年%m月%d日 %H:%M:%S')

class Ubike(RootModel):
    root:list[Station]

ubikes:Ubike = Ubike.model_validate_json(content)
ubikes_list = ubikes.model_dump()
pprint(ubikes_list)

for item in ubikes_list:
    pprint(item,indent=3)